# Inspect EMC data in local DB

## Data availibility

In [1]:
import pandas as pd

import os
from os.path import join
from dotenv import load_dotenv

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from sqlalchemy import create_engine, text

ROOT = None
if ROOT:
    # Load the environment variables from the .env file
    env_file = join(ROOT, '.env')
    load_dotenv(env_file)

    # Get the values of host, user, pswd, db, and schema from the environment variables
    DBHOST = os.getenv('host')
    DBUSER = os.getenv('user')
    DBPSWD = os.getenv('pswd')
    DBNAME = os.getenv('db')
    SCHEMA = 'public'
else:
    DBHOST = 'postgres-1.cvh49u2v99nl.ap-southeast-1.rds.amazonaws.com'
    DBUSER = 'sdcmktops'
    DBPSWD = 'SDCsdc1234'
    DBNAME = 'postgres'
    SCHEMA = 'public'

# Use the values as needed
engine = create_engine(f"postgresql://{DBUSER}:{DBPSWD}@{DBHOST}/{DBNAME}?options=-csearch_path%3D{SCHEMA}", echo=False)

### RealTimeDPR

In [3]:
import pandas as pd

# Assuming 'df' is the dataframe containing the 'RealTimeDPR' table
# Assuming 'engine' is the SQLAlchemy engine object connected to the database
df = pd.read_sql_table('RealTimeDPR', engine)
dates = df['Date'].unique()

In [4]:
date_range = pd.date_range(
    start=dates[0], 
    end=dates[-1], 
    freq='D'
    ).strftime('%Y-%m-%d')

incomplete_dates = []
not_updated_dates = []
for date in date_range:
    count = df[df['Date'] == date].shape[0]
    if count != 48:
        # print(f"Date {date} has {count} records instead of 48")
        incomplete_dates.append(date)
        
    empty_count = df[df["Date"]==date]['CUSEP'].isna().sum()
    if empty_count > 0:
        # print(f"Date {date} has {empty_count} incomplete records")
        not_updated_dates.append(date)


In [8]:
make_up_tasks = set(incomplete_dates + not_updated_dates)
make_up_tasks = list(make_up_tasks)
make_up_tasks.sort()

with open('../tmp/missing_dpr.txt', 'w') as f:
    for task in make_up_tasks:
        f.write(task + '\n')